# Data set Generation

This notebook shows how to generate datasets for the historical validation. We choose a year and get all available phase 4 clinical trials for test and training on this dataset. We additionally export files with positive data for at least a year after the target year. As negative data, we choose all combinations of kinases and cancers that have remained negative up to the present day.

In [1]:
import pandas as pd
import os
import sys
sys.path.insert(0, os.path.abspath('../..'))
from kcet import KcetDatasetGenerator

# Input file. 
We have placed a version of ``clinical_trials_by_phase.tsv`` into the ``data`` subfolder in the ``notebooks`` subfolder of kcet.

In [2]:
ctfile = os.path.abspath(os.path.join('../data', 'clinical_trials_by_phase.tsv'))

## target year
In this example, we show how to extract datasets after the target year until a new target year.

In [3]:
target_year = 2010
num_years_after_the_target_year = 9

In [4]:
dsGen = KcetDatasetGenerator(clinical_trials=ctfile)

[INFO] Reading protein kinase information from /Users/ravanv/PycharmProjects/KCET/input/prot_kinase.tsv
[INFO] ingested symbol_to_id_map with 522 entries such as {'NCBIGene:2870': 'GRK6'}
[INFO] Ingested mesh_id list with 698 entries such as 'meshd000008' and 'meshd000069293', 
[INFO] Ingested _meshid2disease_map with 698 entries
[INFO] Ingested meshid2disease_map with 514 entries
[INFO] Ingested pki_to_kinase with 76 entries
[INFO] Parsed data for the following medications:
[INFO] ibrutinib
[INFO] tofacitinib
[INFO] cobimetinib
[INFO] lenvatinib
[INFO] lorlatinib
[INFO] imatinib
[INFO] nilotinib
[INFO] trastuzumab
[INFO] vandetanib
[INFO] pemigatinib
[INFO] tivozanib
[INFO] dabrafenib
[INFO] osimertinib
[INFO] dasatinib
[INFO] fedratinib
[INFO] crizotinib
[INFO] erdafitinib
[INFO] zanubrutinib
[INFO] tucatinib
[INFO] pertuzumab
[INFO] amlexanox
[INFO] entrectinib
[INFO] fostamatinib
[INFO] alectinib
[INFO] regorafenib
[INFO] icotinib
[INFO] nintedanib
[INFO] ponatinib
[INFO] vemurafen

In [5]:
positive_train_df, negative_train_df, positive_validation_df, negative_validation_df = \
   dsGen.get_data_after_target_year_upto_later_year(target_year=target_year, num_years_later= num_years_after_the_target_year)

Skipping random link(ncbigene2475,meshd058249) since we found it in the positive set
Skipping random link(ncbigene2064,meshd013120) since we found it in the positive set
Skipping random link(ncbigene2322,meshd007822) since we found it in the positive set
Skipping random link(ncbigene147746,meshd018288) since we found it in the positive set
Skipping random link(ncbigene9748,meshd007890) since we found it in the positive set
Skipping random link(ncbigene9748,meshd008223) since we found it in the positive set
Skipping random link(ncbigene2042,meshd009423) since we found it in the positive set
Skipping random link(ncbigene1436,meshd016066) since we found it in the positive set
Skipping random link(ncbigene2042,meshd010255) since we found it in the positive set
Skipping random link(ncbigene9263,meshd018269) since we found it in the positive set
Skipping random link(ncbigene3791,meshd008228) since we found it in the positive set
Skipping random link(ncbigene25,meshd007945) since we found it 

# Positive training data
We use this data frame for test and training. It contains all phase 4 trials completed up to and including the target year.

In [6]:
positive_train_df.head()

,cancer,mesh_id,kinase,gene_id,pki,nct,phase,year
140,Colorectal Neoplasms,meshd015179,EGFR,ncbigene1956,cetuximab,NCT00327093;NCT01564810;NCT01315990,Phase 4,2006
145,Neoplasm Metastasis,meshd009362,EGFR,ncbigene1956,cetuximab,NCT00327093;NCT01564810;NCT00510627,Phase 4,2006
146,"Neoplasms, Second Primary",meshd016609,EGFR,ncbigene1956,cetuximab,NCT00327093,Phase 4,2006
147,Liver Neoplasms,meshd008113,EGFR,ncbigene1956,cetuximab,NCT01564810;NCT00510627,Phase 4,2006
249,"Carcinoma, Non-Small-Cell Lung",meshd002289,EGFR,ncbigene1956,erlotinib,NCT01287754;NCT01320501;NCT01230710;NCT0106688...,Phase 4,2004


In [7]:
#We write the file for machine learning as follows
pos_train_df = positive_train_df[['gene_id','mesh_id']]
outname = "positive_training_upto_{}.tsv".format(target_year)
pos_train_df.to_csv(outname, sep='\t')
print(pos_train_df.head())
print("Number of positive trainig links: {}".format(pos_train_df.shape[0]) )

          gene_id      mesh_id
140  ncbigene1956  meshd015179
145  ncbigene1956  meshd009362
146  ncbigene1956  meshd016609
147  ncbigene1956  meshd008113
249  ncbigene1956  meshd002289
Number of positive trainig links: 161


# Negative training data
This data is used for test and training. It contains a sample with factor times (default=10) as many random combinations of kinases and cancers that we not positive as of the target year. Note that the dataframes for negative examples just have the columns ``gene_id`` and ``mesh_id``.

In [8]:
print(negative_train_df.head())
outname = "negative_training_upto_{}.tsv".format(target_year)
negative_train_df.to_csv(outname, sep='\t')
print("Number of negative trainig links: {}".format(negative_train_df.shape[0]) )

       mesh_id        gene_id
0  meshd009364  ncbigene10290
1  meshd018205  ncbigene27347
2  meshd018222   ncbigene5871
3  meshd015831  ncbigene55781
4  meshd010048   ncbigene5563
Number of negative trainig links: 1350


# Positive validation data
This file has data for all four study phases and for all years following the target year. We should predicting them
all at once and then use postprocessing in a script to figure out the results for each year.

In [9]:
positive_validation_df.head(n=1000)

,mesh_id,gene_id
0,meshd058922,ncbigene2066
1,meshd054219,ncbigene5894
2,meshd012509,ncbigene4914
3,meshd001752,ncbigene3932
4,meshd002277,ncbigene55589
...,...,...
995,meshd046152,ncbigene5894
996,meshd002289,ncbigene2260
997,meshd014594,ncbigene4916
998,meshd018263,ncbigene3815


In [10]:
pos_validation_df = positive_validation_df[['gene_id','mesh_id']]
outname = "positive_validation_{}_years_after_{}.tsv".format(num_years_after_the_target_year,target_year)
pos_validation_df.to_csv(outname, sep='\t')
print(pos_validation_df.head())
print("Number of positive validation links: {}".format(pos_validation_df.shape[0]) )

         gene_id      mesh_id
0   ncbigene2066  meshd058922
1   ncbigene5894  meshd054219
2   ncbigene4914  meshd012509
3   ncbigene3932  meshd001752
4  ncbigene55589  meshd002277
Number of positive validation links: 4794


# Negative validation data
This is similar to the negative training data in that it just has two columns.

In [11]:
print(negative_validation_df.head())
outname = "negative_validation_{}_years_after_{}.tsv".format(num_years_after_the_target_year,target_year)
negative_validation_df.to_csv(outname, sep='\t')
print("Number of negative validation links: {}".format(negative_validation_df.shape[0]) )

          mesh_id        gene_id
0     meshd007340  ncbigene65268
1     meshd010310   ncbigene3791
2     meshd009078   ncbigene2870
3  meshd000071960   ncbigene2044
4  meshd000086002  ncbigene22853
Number of negative validation links: 1350
